In [44]:
# ==============================================================================
# HÜCRE 1: Gerekli Kütüphanelerin Kurulumu
# ==============================================================================
# Projenin çalışması için gerekli olan tüm kütüphaneleri tek bir hücrede topluyoruz.
# Bu, projenin bağımlılıklarını en başta net bir şekilde görmemizi sağlar.
!pip install --upgrade --quiet langchain langchain-core langchain-community langchain-google-genai google-generativeai chromadb PyMuPDF langchain-text-splitters sentence-transformers

In [45]:
# HÜCRE 2: API Anahtarı ve Ortam Değişkenlerinin Ayarlanması
# ==============================================================================
import os
from google.colab import userdata
import google.generativeai as genai

# Google Colab'in "Secrets" özelliğini kullanarak API anahtarımızı güvenli bir şekilde yüklüyoruz.
# Bu yöntem, API anahtarının kod içerisinde görünmesini engelleyerek güvenliği artırır.
os.environ['GOOGLE_API_KEY'] = userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=os.environ['GOOGLE_API_KEY'])

print("✅ Google Generative AI API Anahtarı başarıyla ayarlandı!")

✅ Google Generative AI API Anahtarı başarıyla ayarlandı!


In [46]:
# HÜCRE 3: Veri Kaynağının (PDF) Yüklenmesi ve Metin Olarak Okunması
# ==============================================================================
import requests
import fitz  # PyMuPDF kütüphanesi

# Bilgi kaynağımız olan TFF'nin 2024-2025 Futbol Oyun Kuralları PDF'inin URL'si.
pdf_url = "https://www.tff.org/Resources/TFF/Documents/MHK/2024-2025/oyun-kurallari.pdf"

print("PDF dosyası TFF web sitesinden indiriliyor...")

# requests.get ile PDF dosyasını indiriyoruz.
# UYARI: `verify=False` parametresi SSL sertifika kontrolünü devre dışı bırakır.
# Bu, TFF sitesinin sertifikasıyla ilgili olası bir sorunu aşmak için kullanılmış olabilir.
# Ancak genel projelerde güvenlik açısından önerilmez.
response = requests.get(pdf_url, verify=False)

# HTTP isteğinin başarılı olup olmadığını kontrol ediyoruz (Status Code 200 = Başarılı).
if response.status_code == 200:
    # İndirilen içeriği fitz (PyMuPDF) ile bir PDF dökümanı olarak açıyoruz.
    pdf_document = fitz.open(stream=response.content, filetype="pdf")

    # PDF'in tüm sayfalarındaki metinleri birleştirmek için boş bir string oluşturuyoruz.
    pdf_text = ""
    for page in pdf_document:
        pdf_text += page.get_text()

    print(f"✅ PDF metni başarıyla çıkarıldı! Toplam karakter sayısı: {len(pdf_text)}")
else:
    print(f"❌ PDF indirilemedi. Hata kodu: {response.status_code}")

PDF dosyası TFF web sitesinden indiriliyor...


/usr/local/lib/python3.12/dist-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.tff.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


✅ PDF metni başarıyla çıkarıldı! Toplam karakter sayısı: 195324


In [47]:
# HÜCRE 4: Metnin Anlamlı Parçalara (Chunks) Bölünmesi
# ==============================================================================
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Büyük metinleri, dil modellerinin (LLM) işleyebileceği daha küçük parçalara bölüyoruz.
# RecursiveCharacterTextSplitter, metni anlamsal bütünlüğü koruyarak bölmeye çalışır.
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=2000,      # Her bir metin parçasının maksimum karakter sayısı.
    chunk_overlap=250     # Parçalar arasında anlam bütünlüğünü korumak için bırakılan ortak karakter sayısı.
)

# Metni, belirlenen kurallara göre belgelere (dökümanlara) ayırıyoruz.
docs = text_splitter.create_documents([pdf_text])

print(f"✅ Metin, {len(docs)} adet parçaya bölündü.")

✅ Metin, 112 adet parçaya bölündü.


In [49]:
# ==============================================================================
# HÜCRE 5: Vektör Veritabanı (Vector Store) Oluşturulması
# ==============================================================================
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_community.vectorstores import Chroma
import shutil
import os # os modülünü import ettiğimizden emin olalım

# --- 1. Embedding Modelini Yükleme ---
# Metin parçalarını anlamsal olarak temsil eden sayısal vektörlere (embeddings) dönüştürecek
# olan Google'ın 'text-embedding-004' modelini yüklüyoruz.
print("Embedding modeli yükleniyor...")
embeddings = GoogleGenerativeAIEmbeddings(model="models/text-embedding-004")
print("✅ Embedding modeli başarıyla yüklendi.")


# --- 2. Eski Veritabanını Temizleme ve Yeni Dizini Hazırlama ---
# YOL GÜNCELLEMESİ: Colab için tam dosya yolu belirterek belirsizliği ortadan kaldırıyoruz.
db_directory = "/content/vektor_veritabani"

# Kod her çalıştığında temiz bir başlangıç yapmak için eski veritabanı klasörünü siliyoruz.
if os.path.exists(db_directory):
    print("\nÖnceki veritabanı kalıntıları temizleniyor...")
    shutil.rmtree(db_directory)
    print("✅ Temizlik tamamlandı.")

# EK GÜVENCE: Chroma'ya bırakmadan önce dizinin var olduğundan emin oluyoruz.
# Bu adım, olası izin hatalarını önlemeye yardımcı olabilir.
os.makedirs(db_directory, exist_ok=True)


# --- 3. Yeni Vektör Veritabanını Oluşturma ---
# ChromaDB kullanarak, metin parçalarını (docs) embedding modeli ile vektörlere dönüştürüp
# kalıcı bir veritabanına kaydediyoruz. Bu, anlamsal arama yapmamızı sağlayacak.
print("\nYeni vektör veritabanı oluşturuluyor...")
vector_store = Chroma.from_documents(
    documents=docs,
    embedding=embeddings,
    persist_directory=db_directory  # Güncellenmiş tam yolu kullanıyoruz
)
print("✅ Vektör veritabanı başarıyla oluşturuldu.")


# --- 4. Retriever'ı Yapılandırma ---
# Bu veritabanı üzerinde anlamsal arama yapacak olan "retriever" objesini oluşturuyoruz.
# `k=4` parametresi, bir arama yapıldığında en ilgili 4 metin parçasını getirmesini söyler.
retriever = vector_store.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 4}
)
print("\n✅ Retriever (Bilgi Getirici) başarıyla yapılandırıldı!")

Embedding modeli yükleniyor...
✅ Embedding modeli başarıyla yüklendi.

Yeni vektör veritabanı oluşturuluyor...
✅ Vektör veritabanı başarıyla oluşturuldu.

✅ Retriever (Bilgi Getirici) başarıyla yapılandırıldı!


In [50]:
# HÜCRE 6: LLM ve RAG Zincirinin (Chain) Kurulması
# ==============================================================================
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_google_genai import ChatGoogleGenerativeAI

print("Dil Modeli (LLM) ve RAG zinciri oluşturuluyor...")

# --- 1. Dil Modelini (LLM) Tanımlama ---
# Cevapları üretecek olan 'gemini-2.0-flash' modelini belirli parametrelerle yapılandırıyoruz.
# temperature: Cevapların ne kadar "yaratıcı" olacağını belirler (düşük = daha tutarlı).
llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    temperature=0.6,
    top_k=40,
    max_output_tokens=2048
)

# --- 2. Prompt Şablonunu Oluşturma ---
# LLM'e nasıl davranması gerektiğini, hangi formatta cevap vermesi gerektiğini ve
# hangi bilgileri kullanması gerektiğini söyleyen detaylı bir komut (prompt) şablonu.
template = """
# GÖREVİN
Sen, Uluslararası Futbol Birliği Kurulu'nun (IFAB) resmi 2024-2025 Futbol Oyun Kuralları kitabısın. Senin tek bilgi kaynağın, sana aşağıda "EK BİLGİ" bölümünde verilen metin parçalarıdır. Cevapların net, resmi ve bir kural kitabına yakışır şekilde bilgilendirici olmalıdır.

# CEVAP FORMATI
1.  **Doğrudan Cevap:** Soruya, kuralın en genel ve özet halini vererek başla.
2.  **Kuralın Detayları:** Ardından, "EK BİLGİ"deki ilgili maddeleri kullanarak kuralı adım adım ve detaylı bir şekilde açıkla.
3.  **Kural Dışı Bilgi:** Eğer "EK BİLGİ" içinde sorunun cevabı yoksa, spekülasyon yapma ve kesin bir dille 'Bu konu, sağlanan kural metinlerinde yer almamaktadır.' yanıtını ver.

# BİLGİ KAYNAĞI
EK BİLGİ:
{context}

# KULLANICI SORUSU
SORU: {question}

# DETAYLI CEVAP
"""
prompt = PromptTemplate.from_template(template)

# --- 3. RAG Zincirini Kurma (LCEL ile) ---
# LangChain Expression Language (LCEL) kullanarak tüm adımları birbirine bağlıyoruz.
# 1. Kullanıcının sorusu retriever'a gider ve ilgili metin parçaları (context) bulunur.
# 2. Bulunan context ve orijinal soru, prompt şablonuna yerleştirilir.
# 3. Hazırlanan bu tam prompt, LLM'e gönderilir.
# 4. LLM'den gelen cevap, StrOutputParser ile temiz bir metin olarak çıkarılır.
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

print("✅ RAG Zinciri başarıyla oluşturuldu!")

Dil Modeli (LLM) ve RAG zinciri oluşturuluyor...
✅ RAG Zinciri başarıyla oluşturuldu!


In [51]:
# Oluşturduğumuz RAG zincirine sorumuzu sorarak sistemi test ediyoruz.
soru = "Oyundan ihraç edilmek nedir"

# `invoke` metodu ile zinciri çalıştırıyoruz.
cevap = rag_chain.invoke(soru)

# Sonucu ekrana yazdırıyoruz.
print(f"Soru: {soru}\n")
print(f"Cevap:\n{cevap}")

Soru: Oyundan ihraç edilmek nedir

Cevap:
1.  **Doğrudan Cevap:** Oyundan ihraç edilmek, bir oyuncunun belirli ihlallerde bulunması durumunda oyundan çıkarılmasıdır.

2.  **Kuralın Detayları:**

Bir oyuncu, yedek oyuncu veya değiştirilmiş oyuncu, aşağıdaki ihlallerden birini yaparsa oyundan ihraç edilir:

*   Kasten topla elle oynama ihlaliyle rakip takımın bir golünü veya bariz gol atma şansını önlemek (kendi takımının ceza alanındaki kaleci hariç)
*   Kendi ceza alanı dışında kasıtlı olmayan elle oynama ihlali yaparak rakip takımın bir golünü veya bariz gol atma şansını engellemek
*   genel hareketlenmesi ihlali yapan oyuncunun kalesine doğru olan rakip oyuncunun bir golünü veya bariz gol atma şansını serbest vuruşla cezalandırılması gereken bir ihlal ile önlemek (aşağıda belirtilen durumlar hariç)
*   ciddi faullü oyun
*   birini ısırmak veya birine tükürmek
*   şiddetli hareket
*   saldırgan, hakaret edici veya küfürlü bir şekilde konuşmak ve/veya hareket(ler)de bulunmak
*   aynı m

In [52]:
!pip freeze > requirements.txt